In [1]:
import pymysql
import requests

### 下載分析pm2.5 opendata 資料

In [19]:
url = "https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [20]:
resp=requests.get(url,verify=False)  # verify=False 關閉驗證
resp

c:\Users\USER\Desktop\Django\Class 9_20250820\pm25_tomysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [21]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '14',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '4',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '11',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '3',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '7',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '6',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '10',
  'datacreationdate'

In [7]:
len(datas)

1000

In [22]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '14',
 'datacreationdate': '2025-08-27 20:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [17]:
tablestr="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20)
)
"""
print(tablestr)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20)
)



### 連結資料庫

In [ ]:
# 本地資料庫用
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    port=3307,
    database="demo"
)

print(conn)

In [2]:
# 雲端資料庫用  https://aiven.io/
conn = pymysql.connect(
    host="mysql-63cdab1-aa8954158-dd04.j.aivencloud.com",
    user="avnadmin",
    password="AVNS_Ukq6fXxTr3Q0YUnS8PX",
    port=24112,
    database="defaultdb"
)

print(conn)

In [3]:
cursor=conn.cursor()
cursor

In [18]:
cursor.execute(tablestr)
conn.commit()

### 寫入一筆資料

In [40]:
sqlstr="insert into pm25 (site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3']


"insert ingore into pm25 (site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',4,'2025-08-20 19:00','μg/m3')"

In [21]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入全部資料

In [23]:
# %s 是佔位符
sqlstr="insert ingore into pm25 (site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 抓取pm25 欄位不為空值
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
values

[['員林', '彰化縣', '14', '2025-08-27 20:00', 'μg/m3'],
 ['富貴角', '新北市', '4', '2025-08-27 20:00', 'μg/m3'],
 ['麥寮', '雲林縣', '11', '2025-08-27 20:00', 'μg/m3'],
 ['關山', '臺東縣', '3', '2025-08-27 20:00', 'μg/m3'],
 ['馬公', '澎湖縣', '7', '2025-08-27 20:00', 'μg/m3'],
 ['金門', '金門縣', '6', '2025-08-27 20:00', 'μg/m3'],
 ['埔里', '南投縣', '10', '2025-08-27 20:00', 'μg/m3'],
 ['復興', '高雄市', '6', '2025-08-27 20:00', 'μg/m3'],
 ['永和', '新北市', '3', '2025-08-27 20:00', 'μg/m3'],
 ['竹山', '南投縣', '12', '2025-08-27 20:00', 'μg/m3'],
 ['中壢', '桃園市', '8', '2025-08-27 20:00', 'μg/m3'],
 ['三重', '新北市', '7', '2025-08-27 20:00', 'μg/m3'],
 ['冬山', '宜蘭縣', '7', '2025-08-27 20:00', 'μg/m3'],
 ['宜蘭', '宜蘭縣', '3', '2025-08-27 20:00', 'μg/m3'],
 ['陽明', '臺北市', '0', '2025-08-27 20:00', 'μg/m3'],
 ['花蓮', '花蓮縣', '0', '2025-08-27 20:00', 'μg/m3'],
 ['臺東', '臺東縣', '4', '2025-08-27 20:00', 'μg/m3'],
 ['恆春', '屏東縣', '7', '2025-08-27 20:00', 'μg/m3'],
 ['潮州', '屏東縣', '10', '2025-08-27 20:00', 'μg/m3'],
 ['屏東', '屏東縣', '21', '2025-08-27 20:00', 'μg

In [32]:
cursor.executemany(sqlstr,values)
conn.commit()

In [ ]:
# 關閉資料庫,刪除表格
conn.close()

### 重建資料表做資料約束(重複資料不寫入)
- 重新執行sql語法

In [42]:
# 加上unique key site_time (site,datacreationdate)
# 代表加上不可重複欄位 -> site_time,資料由site,datacreationdate 兩個欄位而來
tablestr="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(tablestr)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



In [56]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    port=3307,
    database="demo"
)

print(conn)

In [57]:
cursor=conn.cursor()
cursor

In [58]:
cursor.execute(tablestr)
conn.commit()

In [59]:
# %s 是佔位符
# inser into 中間加上ignore 會忽略資料庫已經存在的資料
sqlstr="insert ignore into pm25 (site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 抓取pm25 欄位不為空值
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
values

[['員林', '彰化縣', '5', '2025-08-20 20:00', 'μg/m3'],
 ['大城', '彰化縣', '4', '2025-08-20 20:00', 'μg/m3'],
 ['富貴角', '新北市', '3', '2025-08-20 20:00', 'μg/m3'],
 ['麥寮', '雲林縣', '3', '2025-08-20 20:00', 'μg/m3'],
 ['關山', '臺東縣', '10', '2025-08-20 20:00', 'μg/m3'],
 ['馬公', '澎湖縣', '3', '2025-08-20 20:00', 'μg/m3'],
 ['馬祖', '連江縣', '17', '2025-08-20 20:00', 'μg/m3'],
 ['埔里', '南投縣', '2', '2025-08-20 20:00', 'μg/m3'],
 ['復興', '高雄市', '5', '2025-08-20 20:00', 'μg/m3'],
 ['永和', '新北市', '9', '2025-08-20 20:00', 'μg/m3'],
 ['竹山', '南投縣', '0', '2025-08-20 20:00', 'μg/m3'],
 ['中壢', '桃園市', '7', '2025-08-20 20:00', 'μg/m3'],
 ['三重', '新北市', '11', '2025-08-20 20:00', 'μg/m3'],
 ['冬山', '宜蘭縣', '7', '2025-08-20 20:00', 'μg/m3'],
 ['宜蘭', '宜蘭縣', '6', '2025-08-20 20:00', 'μg/m3'],
 ['陽明', '臺北市', '4', '2025-08-20 20:00', 'μg/m3'],
 ['花蓮', '花蓮縣', '6', '2025-08-20 20:00', 'μg/m3'],
 ['臺東', '臺東縣', '8', '2025-08-20 20:00', 'μg/m3'],
 ['恆春', '屏東縣', '6', '2025-08-20 20:00', 'μg/m3'],
 ['潮州', '屏東縣', '7', '2025-08-20 20:00', 'μg/m3

In [60]:
# executemany(語法,資料內容)一次執行多筆資料
cursor.executemany(sqlstr,values)
conn.commit()

In [25]:
conn.close()

### 更新資料
- 再次執行requests重取url
- 並且重跑程式碼
- 將20:00的更新資料寫入

### 驗證資料
- 使用pandas確認資料有更新

In [14]:
import pandas as pd

In [65]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    port=3307,
    database="demo"
)

print(conn)

In [67]:
cursor=conn.cursor()

In [12]:
# 將select 出來的資料,fetchall存到變數
# fetchall一定要接在execute後面
cursor.execute("select * from pm25")

datas=cursor.fetchall()
datas

()

In [24]:
cursor.execute("select * from pm25")

0

In [15]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datacreationdate","itemunit"])
df

,id,site,county,pm25,datacreationdate,itemunit


In [ ]:
# 以城市做分組
# 以取新北市資料為例
df.groupby("county").get_group("新北市")

,id,site,county,pm25,datacreationdate,itemunit
2,3,富貴角,新北市,5,2025-08-20 19:00:00,μg/m3
9,10,永和,新北市,8,2025-08-20 19:00:00,μg/m3
12,13,三重,新北市,11,2025-08-20 19:00:00,μg/m3
62,63,淡水,新北市,4,2025-08-20 19:00:00,μg/m3
63,64,林口,新北市,5,2025-08-20 19:00:00,μg/m3
...,...,...,...,...,...,...
545,947,菜寮,新北市,7,2025-08-20 20:00:00,μg/m3
546,948,新莊,新北市,12,2025-08-20 20:00:00,μg/m3
547,949,板橋,新北市,11,2025-08-20 20:00:00,μg/m3
548,950,土城,新北市,6,2025-08-20 20:00:00,μg/m3


In [ ]:
# drop_duplicates() 刪除重複的資料
df.groupby("county").get_group("新北市").drop_duplicates()

,id,site,county,pm25,datacreationdate,itemunit
2,3,富貴角,新北市,5,2025-08-20 19:00:00,μg/m3
9,10,永和,新北市,8,2025-08-20 19:00:00,μg/m3
12,13,三重,新北市,11,2025-08-20 19:00:00,μg/m3
62,63,淡水,新北市,4,2025-08-20 19:00:00,μg/m3
63,64,林口,新北市,5,2025-08-20 19:00:00,μg/m3
...,...,...,...,...,...,...
545,947,菜寮,新北市,7,2025-08-20 20:00:00,μg/m3
546,948,新莊,新北市,12,2025-08-20 20:00:00,μg/m3
547,949,板橋,新北市,11,2025-08-20 20:00:00,μg/m3
548,950,土城,新北市,6,2025-08-20 20:00:00,μg/m3


In [74]:
len(df)

551

In [ ]:
# 以"site","datacreationdate" 兩個欄位做刪除重複資料
df[["site","datacreationdate"]].drop_duplicates()

,site,datacreationdate
0,員林,2025-08-20 19:00:00
1,大城,2025-08-20 19:00:00
2,富貴角,2025-08-20 19:00:00
3,麥寮,2025-08-20 19:00:00
4,關山,2025-08-20 19:00:00
...,...,...
546,新莊,2025-08-20 20:00:00
547,板橋,2025-08-20 20:00:00
548,土城,2025-08-20 20:00:00
549,汐止,2025-08-20 20:00:00
